In [1]:
from sklearn import model_selection , linear_model, metrics, pipeline,tree, preprocessing,neighbors,ensemble,svm,naive_bayes
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split,RandomizedSearchCV,cross_val_predict
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler

#from sklearn import model_selection import grid_search
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
#from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [30]:
raw_train_data = pd.read_csv('./data/train.csv')
raw_test_data = pd.read_csv('./data/test.csv')

In [31]:
raw_train_data.shape,raw_test_data.shape,891+418
#raw_train_data_shuffled = raw_train_data.reindex(np.random.permutation(raw_train_data.index))

((891, 12), (418, 11), 1309)

In [32]:
#raw_train_data.info()

In [33]:
original_train_data=raw_train_data.copy()

### Train

In [34]:
#Проверка пропусков
NAs = pd.concat([raw_train_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data
Age,177
Cabin,687
Embarked,2


In [35]:
def prepare_data(raw_data,exclude_columns=[]):
    #Обработка пропусков
    #Заполняем медианой
    #raw_data.Age = raw_data.Age.fillna(raw_data.Age.median())
    #Заполняем медианой c группировкой
    grp = raw_data.groupby(['Sex', 'Pclass'])  
    raw_data.Age = grp.Age.apply(lambda x: x.fillna(x.median()))
    #Заполняем модой
    #raw_data.Embarked = raw_data.Embarked.fillna(raw_data.Embarked.value_counts().idxmax())
    raw_data.Embarked.fillna(raw_data.Embarked.mode()[0], inplace = True)
    raw_data.Fare.fillna(raw_data.Fare.mode()[0],inplace = True)
    #Заполняем 'N'
    raw_data.Cabin = raw_data.Cabin.fillna('NA')

    #raw_data.Embarked.value_counts()
    #добавим новые признаки
    raw_data['Family'] = raw_data.Parch + raw_data.SibSp+1
    raw_data['Family_Size'] = pd.cut(raw_data.Family, [0,1.5,4.5,15],labels=['Single','SmallFamily','LargeFamily'])
    #raw_data['Is_Alone'] = (raw_data.Family == 0).astype(int)
    raw_data['Salutation'] = raw_data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
    raw_data['Age_Range'] = pd.cut(raw_data.Age, [0, 10, 20, 30, 40, 50, 60,70,80])
    raw_data['Fare_Category'] = pd.cut(raw_data.Fare, bins=[0,7.90,14.45,31.28,120.0,513.0],include_lowest = True, labels=['Low','Mid','High_Mid','High','Upper_High'])
    raw_data['isCabin']=raw_data.Cabin.apply(lambda x: 1 if x != 'NA' else 0)
    np.seterr(divide = 'ignore')
    raw_data['Log_Fare']=np.log10(raw_data.Fare).replace(-np.inf, 0)
    #raw_train_data.iloc[np.where(raw_train_data.Log_Fare.values<np.finfo(np.float64).min)]
    #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], bins=[0,7.90,14.45,31.28,120.0], labels=['Low','Mid','High_Mid','High'])
    #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], [0,7.90,14.45,31.28,120.0,513.0],include_lowest = True)
    #raw_data.Salutation.nunique()
    #####################New features#####################
    name_dict = {"Capt":       "officer",
                 "Col":        "officer",
                 "Major":      "officer",
                 "Dr":         "officer",
                 "Rev":        "officer",
                 "Jonkheer":   "snob",
                 "Don":        "snob",
                 "Sir" :       "snob",
                 "the Countess":"snob",
                 "Dona":       "snob",
                 "Lady" :      "snob",
                 "Mme":        "married",
                 "Ms":         "married",
                 "Mrs" :       "married",
                 "Miss" :      "single",
                 "Mlle":       "single",
                 "Mr" :        "man",
                 "Master" :    "boy"
                }
    raw_data['Salutation_type'] = raw_data['Salutation'].map(name_dict)
    raw_data.drop(exclude_columns, axis = 1, inplace = True)
    
    return 0


In [36]:
prepare_data(raw_train_data,['Age','Fare','Name','Sex'])

0

In [39]:
raw_train_data.head()
#display_all(df.describe(include='all').T)

,PassengerId,Survived,Pclass,SibSp,Parch,Ticket,Cabin,Embarked,Family,Family_Size,Salutation,Age_Range,Fare_Category,isCabin,Log_Fare,Salutation_type
0,1,0,3,1,0,A/5 21171,NA,S,2,SmallFamily,Mr,"(20, 30]",Low,0,0.860338,man
1,2,1,1,1,0,PC 17599,C85,C,2,SmallFamily,Mrs,"(30, 40]",High,1,1.852988,married
2,3,1,3,0,0,STON/O2. 3101282,NA,S,1,Single,Miss,"(20, 30]",Mid,0,0.898999,single
3,4,1,1,1,0,113803,C123,S,2,SmallFamily,Mrs,"(30, 40]",High,1,1.725095,married
4,5,0,3,0,0,373450,NA,S,1,Single,Mr,"(30, 40]",Mid,0,0.905796,man


In [40]:
#Проверка пропусков
NAs = pd.concat([raw_train_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data


In [41]:
#raw_train_data['Singleton']=raw_train_data['Family'].apply(lambda x: 1 if x == 1 else 0)
#raw_train_data['SmallFamily']=raw_train_data['Family'].apply(lambda x: 1 if 2<=x <= 4 else 0)
#raw_train_data['LargeFamily']=raw_train_data['Family'].apply(lambda x: 1 if 5<=x  else 0)
#Удалим исходные, которые были базой
#raw_train_data.drop(['SibSp','Parch','Age','Fare'], axis = 1, inplace = True)
#raw_train_data.isnull().any()

In [42]:
train_labels = raw_train_data['Survived']
train_data = raw_train_data.drop(['PassengerId', 'Survived'], axis = 1)

In [43]:
string_data_columns = ['Name','Ticket','Cabin']
categorical_data_columns = ['Pclass','Sex','Embarked','Salutation','Age_Range', 'Fare_Category','Family_Size','Salutation_type']
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare','Family','isCabin','Log_Fare']
#numeric_data_columns = ['Family']

In [44]:
set(list(train_data.columns))==set(string_data_columns+categorical_data_columns+numeric_data_columns)

False

In [45]:
#Оставшиеся колонки
set(train_data.columns) - set(numeric_data_columns)-set(string_data_columns)-set(categorical_data_columns)

set()

In [46]:
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)

In [49]:
train_data.head()

,Pclass,SibSp,Parch,Ticket,Cabin,Embarked,Family,Family_Size,Salutation,Age_Range,Fare_Category,isCabin,Log_Fare,Salutation_type
0,3,1,0,A/5 21171,NA,S,2,SmallFamily,Mr,"(20, 30]",Low,0,0.860338,man
1,1,1,0,PC 17599,C85,C,2,SmallFamily,Mrs,"(30, 40]",High,1,1.852988,married
2,3,0,0,STON/O2. 3101282,NA,S,1,Single,Miss,"(20, 30]",Mid,0,0.898999,single
3,1,1,0,113803,C123,S,2,SmallFamily,Mrs,"(30, 40]",High,1,1.725095,married
4,3,0,0,373450,NA,S,1,Single,Mr,"(30, 40]",Mid,0,0.905796,man


### Test 

In [60]:
#Проверка пропусков
NAs = pd.concat([raw_test_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]


,data
Age,86
Fare,1
Cabin,327


In [61]:
prepare_data(raw_test_data)

0

In [62]:
#Проверка пропусков
NAs = pd.concat([raw_test_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data


In [63]:
test_data = raw_test_data.drop(['PassengerId'], axis = 1)
#Оставшиеся колонки
set(test_data.columns) - set(numeric_data_columns)-set(string_data_columns)-set(categorical_data_columns)

set()

In [64]:
#test_data.isnull().any()

## Все колонки (лин завис)

In [65]:
#classifier = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

In [66]:
def create_estimator(numeric_data_indices,categorical_data_indices,classifier):
    estimator = pipeline.Pipeline(steps = [       
        (
            'feature_processing', pipeline.FeatureUnion(transformer_list = [        
                #binary
                #('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                        
                #numeric
                ('numeric_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices],validate=False)),
                    ('scaling', preprocessing.StandardScaler())            
                            ])),
        
                #categorical
                ('categorical_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices],validate=False)),
                    ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore',sparse=False))            
                            ])),
            ])
        ),
        ('model_fitting', classifier)
        ]
    )
    return estimator

In [67]:
#cross_val_score(estimator, train_data.values, train_labels, scoring = 'accuracy', cv = 3)
#estimator.fit(train_data.values, train_labels)
#train_labels = raw_train_data['Survived']

### Отбор признаков

In [163]:
#classifier_dict = {'RiC':linear_model.RidgeClassifier(),'RF':ensemble.RandomForestClassifier(n_estimators=10),'GB':ensemble.GradientBoostingClassifier()}
classifier_dict = {'RiC':linear_model.RidgeClassifier(),'SVC':svm.SVC(gamma='auto'),'DeTree':tree.DecisionTreeClassifier()}

In [164]:
all_column=[[],
    #По одной
    ['Name'],['Ticket'],['Cabin'],['Pclass'],['Sex'],['Embarked'],['Salutation'],['Age_Range'], ['Fare_Category'],['Family_Size'],['Salutation_type'],
            ['Age'], ['SibSp'], ['Parch'], ['Fare'],['Family'],['isCabin'],['Log_Fare'],
    #Сочетания (основная с производной колонкой)
    ['Age','Age_Range'],['Fare','Fare_Category'],['Fare','Fare_Category','Log_Fare'],['Cabin','isCabin'],
           ]

In [165]:
exclude_col_list=[]
clf_res_list=[]
#cv_res_list=[]
mean_res_list=[]
max_res_list=[]
min_res_list=[]
std_res_list=[]
#test_acc=[]
for exclude_col in all_column:
    print(exclude_col)
    current_train_data=original_train_data.copy()
    prepare_data(current_train_data,exclude_col)
    #print(current_train_data.columns)
    current_categorical_data_indices = np.array([(column in categorical_data_columns) for column in current_train_data.columns], dtype = bool)
    current_numeric_data_indices = np.array([(column in numeric_data_columns) for column in current_train_data.columns], dtype = bool)
    
    for k, v in classifier_dict.items():
        #print("Code : {0}, Value : {1}".format(k, v))
        print('***Classifier*****',k)
        clf=create_estimator(current_numeric_data_indices,current_categorical_data_indices,v)
        clf_scoring = cross_val_score(clf, current_train_data.values, train_labels, scoring = 'accuracy', cv = 3)
            #print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), clf_scoring.min(), clf_scoring.std()))
        exclude_col_list.append(exclude_col)   
        clf_res_list.append(k)
        #cv_res_list.append(name)
        mean_res_list.append(clf_scoring.mean())
        max_res_list.append(clf_scoring.max())
        min_res_list.append(clf_scoring.min())
        std_res_list.append(clf_scoring.std())
        #clf.fit(X_train, y_train)
        #y_pred=clf.predict(X_test)
        #test_acc.append(accuracy_score(y_test, y_pred))
    

[]
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Name']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Ticket']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Cabin']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Pclass']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Sex']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Embarked']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Salutation']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Age_Range']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Fare_Category']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Family_Size']
***Classifier***** RiC
***Classifier***** SVC
***Classifier***** DeTree
['Salutation_type']
***Classifier***** RiC
***Classifier***** SVC
***Classifier*

In [166]:
select_f_df=pd.DataFrame({'ex_col':exclude_col_list,'clf':clf_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list})
select_f_df

,ex_col,clf,mean,max,min,std
0,[],RiC,0.824916,0.831650,0.814815,0.007274
1,[],SVC,0.835017,0.841751,0.821549,0.009523
2,[],DeTree,0.773288,0.801347,0.734007,0.028614
3,[Name],RiC,0.824916,0.831650,0.814815,0.007274
4,[Name],SVC,0.835017,0.841751,0.821549,0.009523
...,...,...,...,...,...,...
64,"[Fare, Fare_Category, Log_Fare]",SVC,0.832772,0.841751,0.821549,0.008399
65,"[Fare, Fare_Category, Log_Fare]",DeTree,0.782267,0.818182,0.747475,0.028877
66,"[Cabin, isCabin]",RiC,0.820426,0.828283,0.808081,0.008837
67,"[Cabin, isCabin]",SVC,0.835017,0.841751,0.821549,0.009523


In [167]:
select_f_df.nlargest(10, 'mean')

,ex_col,clf,mean,max,min,std
1,[],SVC,0.835017,0.841751,0.821549,0.009523
4,[Name],SVC,0.835017,0.841751,0.821549,0.009523
7,[Ticket],SVC,0.835017,0.841751,0.821549,0.009523
10,[Cabin],SVC,0.835017,0.841751,0.821549,0.009523
19,[Embarked],SVC,0.835017,0.841751,0.821549,0.009523
28,[Fare_Category],SVC,0.835017,0.841751,0.821549,0.009523
52,[isCabin],SVC,0.835017,0.841751,0.821549,0.009523
55,[Log_Fare],SVC,0.835017,0.841751,0.821549,0.009523
67,"[Cabin, isCabin]",SVC,0.835017,0.841751,0.821549,0.009523
40,[SibSp],SVC,0.833895,0.841751,0.821549,0.008837


In [153]:
select_f_df.nlargest(5, 'mean')

,ex_col,clf,mean,max,min,std
6,[Embarked],RiC,0.828283,0.838384,0.811448,0.011983
5,[Sex],RiC,0.826038,0.835017,0.814815,0.008399
11,[Salutation_type],RiC,0.826038,0.835017,0.814815,0.008399
0,[],RiC,0.824916,0.831650,0.814815,0.007274
1,[Name],RiC,0.824916,0.831650,0.814815,0.007274


In [145]:
select_f_df.nlargest(5, 'mean',keep='last')

,ex_col,clf,mean,max,min,std
5,[Embarked],RiC,0.828283,0.838384,0.811448,0.011983
10,[Salutation_type],RiC,0.826038,0.835017,0.814815,0.008399
4,[Sex],RiC,0.826038,0.835017,0.814815,0.008399
15,[Family],RiC,0.824916,0.831650,0.814815,0.007274
13,[Parch],RiC,0.824916,0.831650,0.814815,0.007274


In [133]:
select_f_df[select_f_df['mean']==select_f_df['mean'].idxmax()]

,ex_col,clf,mean,max,min,std


In [138]:
select_f_df['mean'].idxmax()

5

In [137]:
select_f_df.iloc[select_f_df['mean'].idxmax()]

ex_col    [Embarked]
clf              RiC
mean        0.828283
max         0.838384
min         0.811448
std        0.0119833
Name: 5, dtype: object

In [127]:
select_f_df

,ex_col,clf,mean,max,min,std
0,[Name],RiC,0.824916,0.831650,0.814815,0.007274
1,[Ticket],RiC,0.824916,0.831650,0.814815,0.007274
2,[Cabin],RiC,0.824916,0.831650,0.814815,0.007274
3,[Pclass],RiC,0.819304,0.824916,0.808081,0.007936
4,[Sex],RiC,0.826038,0.835017,0.814815,0.008399
5,[Embarked],RiC,0.828283,0.838384,0.811448,0.011983
6,[Salutation],RiC,0.823793,0.835017,0.814815,0.008399
7,[Age_Range],RiC,0.824916,0.831650,0.814815,0.007274
8,[Fare_Category],RiC,0.823793,0.831650,0.811448,0.008837
9,[Family_Size],RiC,0.819304,0.831650,0.801347,0.012992


### Разделение выборки

In [68]:
X_train, X_test, y_train, y_test = train_test_split(train_data.values, train_labels, test_size=0.5,shuffle=True, random_state=42)
#stratify=train_labels

In [69]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((445, 14), (446, 14), (445,), (446,))

### оценка алгоритмов

In [70]:
# кросс валидация тратегии
cv_strategy_SKF = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
cv_strategy_KF =model_selection.KFold(n_splits=5, shuffle = True, random_state = 0)
cv_strategy_ShS =model_selection.ShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_strategy_ShS =model_selection.StratifiedShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_dict = {'SKF':cv_strategy_SKF,'KF':cv_strategy_KF,'ShS':cv_strategy_ShS,'ShS':cv_strategy_ShS}

In [71]:
classifier_dict = {'RiC':linear_model.RidgeClassifier(),'SVC':svm.SVC(gamma='auto'),'SGD':linear_model.SGDClassifier(),'LR':linear_model.LogisticRegression(solver='lbfgs'),
                   'KNC':neighbors.KNeighborsClassifier(),'DeTree':tree.DecisionTreeClassifier(),
                   'RF':ensemble.RandomForestClassifier(n_estimators=10),'GB':ensemble.GradientBoostingClassifier(),
                   'BG':ensemble.BaggingClassifier(),'NB':naive_bayes.GaussianNB()}

In [72]:
# кросс валидация оценка

clf_res_list=[]
cv_res_list=[]
mean_res_list=[]
max_res_list=[]
min_res_list=[]
std_res_list=[]
test_acc=[]
for k, v in classifier_dict.items():
    #print("Code : {0}, Value : {1}".format(k, v))
    print('***Classifier*****',k)
    clf=create_estimator(numeric_data_indices,categorical_data_indices,v)
    for name,val in cv_dict.items():
        clf_scoring = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = val)
        print('--cv--',name,'----',val)
        #print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), clf_scoring.min(), clf_scoring.std()))
        clf_res_list.append(k)
        cv_res_list.append(name)
        mean_res_list.append(clf_scoring.mean())
        max_res_list.append(clf_scoring.max())
        min_res_list.append(clf_scoring.min())
        std_res_list.append(clf_scoring.std())
        clf.fit(X_train, y_train)
        y_pred=clf.predict(X_test)
        test_acc.append(accuracy_score(y_test, y_pred))

***Classifier***** RiC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SVC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SGD
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** LR
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedSh

In [73]:
pd.DataFrame({'clf':clf_res_list, 'cv':cv_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list,'test_acc':test_acc})

,clf,cv,mean,max,min,std,test_acc
0,RiC,SKF,0.835964,0.866667,0.800000,0.027916,0.813901
1,RiC,KF,0.817978,0.853933,0.775281,0.026013,0.813901
2,RiC,ShS,0.808929,0.848214,0.794643,0.020045,0.813901
3,SVC,SKF,0.820156,0.866667,0.788889,0.027710,0.822870
4,SVC,KF,0.822472,0.865169,0.764045,0.033633,0.822870
5,SVC,ShS,0.819643,0.839286,0.794643,0.015361,0.822870
6,SGD,SKF,0.811292,0.831461,0.777778,0.018531,0.677130
7,SGD,KF,0.784270,0.831461,0.741573,0.035814,0.769058
8,SGD,ShS,0.789286,0.848214,0.732143,0.038548,0.802691
9,LR,SKF,0.826899,0.877778,0.795455,0.032452,0.811659


### Подбор пареметров для RidgeClassifier

In [70]:
#SGD
#RidgeClassifier

parameters_grid = {
    'model_fitting__alpha' : [1e-3, 1e-2, 1e-1, 1,1e1,1e2], #RC
    #'model_fitting__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #GB
    #'model_fitting__class_weight' : [None,'balanced'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [71]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(fit_intercept=False))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [72]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [73]:
%%time
#grid_cv.fit(train_data.values, train_labels)
grid_cv.fit(X_train, y_train)

print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.8404494382022472
{'model_fitting__alpha': 10.0}
CPU times: user 385 ms, sys: 337 µs, total: 385 ms
Wall time: 201 ms


In [74]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8251121076233184

In [75]:
type(grid_cv.best_estimator_[1])

sklearn.linear_model.ridge.RidgeClassifier

In [76]:
#Коэфициенты модели
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[0][1].fit_transform(X_train)[:2]
#one_hot_cat_col=grid_cv.best_estimator_.steps[0][1].transformer_list[1][1][1].get_feature_names(categorical_data_columns)
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].categories_
all_column=numeric_data_columns+list(grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].get_feature_names())
coef=grid_cv.best_estimator_.steps[1][1].coef_[0]
sorted(zip(np.abs(coef),all_column),reverse=True)

[(0.2583893302999091, 'x3_LargeFamily'),
 (0.2491372890352995, 'x4_Mr'),
 (0.24913728903529506, 'x7_man'),
 (0.24206154069308153, 'x7_boy'),
 (0.2420615406930811, 'x4_Master'),
 (0.21590817942763346, 'x1_female'),
 (0.20942839038800218, 'x1_male'),
 (0.20235264204578535, 'x7_officer'),
 (0.18392780433245198, 'x3_Single'),
 (0.17017557802058017, 'x7_married'),
 (0.1594704574651939, 'x6_Upper_High'),
 (0.15775405090002434, 'x0_2'),
 (0.15677986854447074, 'isCabin'),
 (0.15424661890167843, 'x4_Rev'),
 (0.1489753252589298, 'x4_Mrs'),
 (0.14428835857498598, 'x5_(60, 70]'),
 (0.12830185103403588, 'x0_3'),
 (0.1269394174675921, 'x2_S'),
 (0.12297527155798361, 'x5_(70, 80]'),
 (0.11197607164586176, 'x6_High_Mid'),
 (0.11078605428364559, 'x2_C'),
 (0.08756993292202067, 'Log_Fare'),
 (0.0830507341846522, 'Age'),
 (0.08094131500708512, 'x3_SmallFamily'),
 (0.07964427282915638, 'x5_(10, 20]'),
 (0.07787376079302431, 'x4_Dr'),
 (0.07780414522617043, 'x4_Col'),
 (0.07723556884565377, 'SibSp'),
 (0.0

### Подбор пареметров для SVC

In [77]:
#SVC

parameters_grid = {
    #'model_fitting__C' : [0.01,0.1, 1, 10, 100,], 
    'model_fitting__C' : [0.8, 0.85, 0.9],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], 
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [78]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [79]:
#clf.get_params().keys()

In [80]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [81]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.8247191011235955
{'model_fitting__C': 0.85, 'model_fitting__class_weight': None, 'model_fitting__decision_function_shape': 'ovo', 'model_fitting__kernel': 'rbf'}
CPU times: user 1.77 s, sys: 0 ns, total: 1.77 s
Wall time: 1.77 s


In [82]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8340807174887892

### Подбор пареметров для GB

In [83]:
#GradientBoosting

parameters_grid = {
    'model_fitting__n_estimators' : [3,4, 8, 16, 32, 64, 100], #GB
    'model_fitting__max_depth' : [3,5,7], #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    'model_fitting__learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 1, 5),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5),
    'model_fitting__max_features':['auto',"log2","sqrt",None],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [84]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier())
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [85]:
#clf.get_params().keys()

In [86]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [87]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

/home/guric/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8471910112359551
{'model_fitting__learning_rate': 0.01, 'model_fitting__max_depth': 7, 'model_fitting__max_features': 'log2', 'model_fitting__n_estimators': 100}
CPU times: user 1min 12s, sys: 0 ns, total: 1min 12s
Wall time: 1min 12s


In [88]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8318385650224215

In [89]:
#y_pred=grid_cv.predict(X_test)

In [90]:
#accuracy_score(y_test, y_pred)

### Подбор пареметров для LR

In [91]:
#LR

parameters_grid = {
    'model_fitting__C' : [0.001,0.01,0.1,1,10,100], #GB
    #'model_fitting__penalty' : ['l2','l1','none'],
    'model_fitting__solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
}

In [92]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(max_iter=1000))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [93]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [94]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.8382022471910112
{'model_fitting__C': 0.1, 'model_fitting__solver': 'newton-cg'}
CPU times: user 1.73 s, sys: 16.1 ms, total: 1.74 s
Wall time: 879 ms


In [95]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8228699551569507

### Подбор пареметров для RF

In [96]:
#RandomForestClassifier(n_estimators=10)

parameters_grid = {
    'model_fitting__n_estimators' : [1, 2, 4, 8, 16, 32, 64, 100, 200,500], #GB
    #'model_fitting__max_depth' : np.linspace(1, 32, 32, endpoint=True), #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    'model_fitting__min_samples_split': [2,5,7,10],
    'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
    #'model_fitting__max_features':["log2","sqrt",'auto'],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
    #'model_fitting__class_weight' : ['balanced',None]
}

In [97]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,ensemble.RandomForestClassifier(class_weight='balanced'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [98]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [99]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [100]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

/home/guric/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.7842696629213484
{'model_fitting__n_estimators': 200, 'model_fitting__min_samples_split': 5, 'model_fitting__min_samples_leaf': 0.1}
CPU times: user 41.3 s, sys: 131 ms, total: 41.5 s
Wall time: 41.4 s


In [101]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.7847533632286996

## Обучение финального классификатора ALL_TRAIN

In [ ]:
#SVC

parameters_grid = {
    'model_fitting__gamma' : ['auto',0.0001,0.001,0.01,0.1,1,10], 
    'model_fitting__C' : [0.01,0.1, 1, 10],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], #GB
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
grid_cv.fit(train_data.values, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

### формирование файла Kaggle

In [ ]:
result = grid_cv.predict(test_data.values)
submission = pd.DataFrame({'PassengerId':raw_test_data.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)


In [ ]:
filename = 'titanic13_SVC.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

In [ ]:
#Преобразование категориальных признаков в отдельном df
#dummy_data=pd.get_dummies(data=raw_test_data, columns=categorical_data_columns)
#dummy_index=dummy_data.columns.str.contains('|'.join(categorical_data_columns))
#dummy_data=dummy_data.loc[:,dummy_index]
#Числовые данные в отдельном df
#numeric_data=raw_test_data[numeric_data_columns]
#создаем стандартный scaler
#scaler = StandardScaler()
#scaled_test_data=scaler.transform(numeric_data)
#scaled_test_data = scaler.transform(test_data)
#all_test_data=np.hstack((scaled_test_data,dummy_data.values))

### Стекинг для валидации

In [102]:
X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(train_data.values, train_labels.values, test_size=0.25, random_state=42)

In [103]:
X_train_st.shape,y_train_st.shape,y_train_st.shape

((668, 18), (668,), (668,))

In [104]:
#linear_model.RidgeClassifier(alpha=10.0)
#svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')
#
clfs = [create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(solver='liblinear',class_weight=None,tol=1e-05, penalty='l2')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(alpha=10.0)),
        create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, max_features='auto', n_estimators= 32))
       ]

In [105]:
dataset_blend_train = np.zeros((X_train_st.shape[0], len(clfs)))

In [106]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
for j, clf in enumerate(clfs):
    print (j, clf[1])
    for i, (train_, test_) in enumerate(kf.split(X_train_st, y_train_st)):
        print ("Fold", i)
        X_tr = X_train_st[train_]
        y_tr = y_train_st[train_]
        X_te = X_train_st[test_]
        y_te = y_train_st[test_]
        clf.fit(X_tr, y_tr)
        print('------------------------',type(clf[1]))
        if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
            print('----------SVM/RidgeCl-------')
            y_submission = clf.predict(X_te)
        else:
            print('----------Other-------')
            y_submission = clf.predict_proba(X_te)[:, 1]
        dataset_blend_train[test_, j] = y_submission

0 SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Fold 0
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 1
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 2
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 3
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 4
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=1e-05, verbos

In [108]:
isinstance(clf[1],svm.SVC),
type(clf[1])

sklearn.ensemble.gradient_boosting.GradientBoostingClassifier

In [109]:
dataset_blend_train.shape

(668, 4)

In [110]:
clf2 = linear_model.RidgeClassifier(class_weight=None,tol=1e-05, random_state=11)
ls = cross_val_score(clf2, dataset_blend_train, y_train_st, cv=5, scoring='accuracy')
print(ls)
np.mean(ls)

[0.82089552 0.8880597  0.80597015 0.83458647 0.84962406]


0.8398271798900236

In [111]:
clf2.fit(dataset_blend_train, y_train_st)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=11, solver='auto',
                tol=1e-05)

In [112]:
#clf2.predict()

### Тест для стекинга

In [113]:
#kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
X_meta = np.zeros((X_test_st.shape[0],len(clfs)))
for j, clf in enumerate(clfs):
    print (j, clf[1])
    #for i, (train_, test_) in enumerate(kf.split(train_data, train_labels)):
    #    print ("Fold", i)
    #    X_tr = train_data.values[train_]
    #    y_tr = train_labels[train_]
    #    X_te = train_data.values[test_]
    #    y_te = train_labels[test_]
    #    clf_pipeline=create_estimator(numeric_data_indices,categorical_data_indices,clf)
    #    clf_pipeline.fit(X_tr, y_tr)
    print('------------------------',type(clf[1]))
    if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
        print('----------SVM/RidgeCl-------')
        #y_submission = clf_.predict(X_te)
        X_meta[:, j] = clf.predict(X_test_st)
    else:
        print('----------Other-------')
        #y_submission = clf_pipeline.predict_proba(X_te)[:, 1]
        X_meta[:, j] = clf.predict_proba(X_test_st)[:, 1]
    #dataset_blend_train[test_, j] = y_submission

0 SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=1e-05, verbose=0,
                   warm_start=False)
------------------------ <class 'sklearn.linear_model.logistic.LogisticRegression'>
----------Other-------
2 RidgeClassifier(alpha=10.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)
------------------------ <class 'sklearn.linear

In [114]:
X_test.shape,X_meta.shape

((446, 18), (223, 4))

In [115]:
pred_meta=clf2.predict(X_meta)

In [116]:
accuracy_score(y_test_st, pred_meta)

0.8161434977578476

## Стекинг для ответа kaggle

In [117]:
#linear_model.RidgeClassifier(alpha=10.0)
#svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')
#
clfs = [create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto',C=0.8, decision_function_shape='ovo', kernel= 'linear')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(solver='liblinear',class_weight=None,tol=1e-05, penalty='l2')),
        create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(alpha=10.0)),
        create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, max_features='auto', n_estimators= 32))
       ]

In [118]:
X_train_st_kaggle=train_data.values
y_train_st_kaggle=train_labels.values

In [119]:
dataset_blend_train = np.zeros((X_train_st_kaggle.shape[0], len(clfs)))

In [120]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
for j, clf in enumerate(clfs):
    print (j, clf[1])
    for i, (train_, test_) in enumerate(kf.split(X_train_st_kaggle, y_train_st_kaggle)):
        print ("Fold", i)
        X_tr = X_train_st_kaggle[train_]
        y_tr = y_train_st_kaggle[train_]
        X_te = X_train_st_kaggle[test_]
        y_te = y_train_st_kaggle[test_]
        clf.fit(X_tr, y_tr)
        print('------------------------',type(clf[1]))
        if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
            print('----------SVM/RidgeCl-------')
            y_submission = clf.predict(X_te)
        else:
            print('----------Other-------')
            y_submission = clf.predict_proba(X_te)[:, 1]
        dataset_blend_train[test_, j] = y_submission

0 SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Fold 0
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 1
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 2
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 3
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
Fold 4
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=1e-05, verbos

In [121]:
clf_meta_kaggle = linear_model.RidgeClassifier(class_weight=None,tol=1e-05, random_state=11)
clf_meta_kaggle.fit(dataset_blend_train, y_train_st_kaggle)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=11, solver='auto',
                tol=1e-05)

### Стекинг test для kaggle

In [122]:
X_test_st_kaggle=test_data.values

In [123]:
#kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
X_meta = np.zeros((X_test_st_kaggle.shape[0],len(clfs)))
for j, clf in enumerate(clfs):
    print (j, clf[1])
    #for i, (train_, test_) in enumerate(kf.split(train_data, train_labels)):
    #    print ("Fold", i)
    #    X_tr = train_data.values[train_]
    #    y_tr = train_labels[train_]
    #    X_te = train_data.values[test_]
    #    y_te = train_labels[test_]
    #    clf_pipeline=create_estimator(numeric_data_indices,categorical_data_indices,clf)
    #    clf_pipeline.fit(X_tr, y_tr)
    print('------------------------',type(clf[1]))
    if isinstance(clf[1],svm.SVC) or isinstance(clf[1],linear_model.ridge.RidgeClassifier):
        print('----------SVM/RidgeCl-------')
        #y_submission = clf_.predict(X_te)
        X_meta[:, j] = clf.predict(X_test_st_kaggle)
    else:
        print('----------Other-------')
        #y_submission = clf_pipeline.predict_proba(X_te)[:, 1]
        X_meta[:, j] = clf.predict_proba(X_test_st_kaggle)[:, 1]
    #dataset_blend_train[test_, j] = y_submission

0 SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
------------------------ <class 'sklearn.svm.classes.SVC'>
----------SVM/RidgeCl-------
1 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=1e-05, verbose=0,
                   warm_start=False)
------------------------ <class 'sklearn.linear_model.logistic.LogisticRegression'>
----------Other-------
2 RidgeClassifier(alpha=10.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)
------------------------ <class 'sklearn.linear

In [124]:
X_meta.shape

(418, 4)

In [125]:
result=clf_meta_kaggle.predict(X_meta)
submission = pd.DataFrame({'PassengerId':raw_test_data.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)

(418, 2)


In [126]:
filename = 'titanic16_stack.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: titanic16_stack.csv


In [ ]:
string_data_columns = ['Name','Ticket','Cabin']
categorical_data_columns = ['Pclass','Sex','Embarked','Salutation','Age_Range', 'Fare_Category','Family_Size','Salutation_type']
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare','Family','isCabin','Log_Fare']
#numeric_data_columns = ['Family']